# Covered Call — Integration Test Notebook

Tests the full CC lifecycle on the paper account:
1. Buy shares of random S&P 500 tickers
2. Detect stock positions via CC manager
3. Select and sell covered calls
4. Monitor active CCs
5. Clean up

**Run cell-by-cell during market hours.**

In [1]:
# Imports
import random
import json
import asyncio
import time

from dotenv import load_dotenv
load_dotenv(override=True)

from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce

from csp.config import StrategyConfig
from csp.clients import AlpacaClientManager
from csp.data.vix import VixDataFetcher
from csp.data.greeks import GreeksCalculator
from csp.data.manager import DataManager
from csp.storage import build_storage_backend
from csp.trading.execution import ExecutionEngine
from csp.trading.risk import RiskManager
from csp.trading.metadata import StrategyMetadataStore
from csp.trading.portfolio import print_portfolio_status
from csp.trading.liquidation import liquidate_all_holdings

from covered_call.config import CoveredCallConfig
from covered_call.store import WheelPositionStore
from covered_call.manager import CoveredCallManager

print("Imports OK")

Imports OK


In [2]:
# Configuration
N_TEST_TICKERS = 3       # how many random S&P 500 stocks to buy
SHARES_PER_TICKER = 100  # must be >= 100 for 1 CC contract

cc_config = CoveredCallConfig(
    enabled=True,
    cc_strike_mode="delta",     # try: "delta", "min_delta", "pct_change", "min_pct_change", "min_daily_return"
    cc_strike_delta=0.30,
    cc_min_dte=1,
    cc_max_dte=10,              # wider range for testing (more contracts available)
    cc_exit_mode="strike_recovery",
)

config = StrategyConfig(
    paper_trading=True,
    entry_order_type="market",   # market orders for reliable fills during testing
    covered_call_config=cc_config,
    print_mode="verbose",
)

print(f"CC config: mode={cc_config.cc_strike_mode}, delta={cc_config.cc_strike_delta}, "
      f"DTE=[{cc_config.cc_min_dte}, {cc_config.cc_max_dte}], exit={cc_config.cc_exit_mode}")
print(f"Universe: {len(config.ticker_universe)} S&P 500 symbols")
print(f"Will buy {SHARES_PER_TICKER} shares of {N_TEST_TICKERS} random tickers")

CC config: mode=delta, delta=0.3, DTE=[1, 10], exit=strike_recovery
Universe: 501 S&P 500 symbols
Will buy 100 shares of 3 random tickers


In [3]:
# Connect & Initialize
alpaca = AlpacaClientManager(paper=config.paper_trading)
vix_fetcher = VixDataFetcher()
greeks_calc = GreeksCalculator()
data_manager = DataManager(alpaca, config)
execution = ExecutionEngine(alpaca, config)
risk_manager = RiskManager(config)

storage = build_storage_backend(config)
metadata = StrategyMetadataStore(path="strategy_metadata.json", backend=storage)

wheel_store = WheelPositionStore(path="wheel_positions_test.json", backend=storage)

cc_manager = CoveredCallManager(
    cc_config=cc_config,
    strategy_config=config,
    store=wheel_store,
    data_manager=data_manager,
    execution=execution,
    risk_manager=risk_manager,
    metadata_store=metadata,
    alpaca_manager=alpaca,
)

current_vix = vix_fetcher.get_current_vix()
print(f"Alpaca connected (paper={alpaca.paper})")
print(f"VIX: {current_vix:.2f}")
print(f"CC Manager ready")

Alpaca connected (paper=True)
VIX: 20.23
CC Manager ready


In [4]:
# Portfolio status before test
print_portfolio_status(alpaca)

Alpaca Account Information
Account status:              AccountStatus.ACTIVE
Cash available:              $1,008,589.96
Buying power (with margin):  $2,990,039.96
Portfolio value:             $1,006,139.96
Options trading level:       3
Trading blocked:             False

Current Positions (4):
  Symbol                    Qty Side       Strike        Entry        Current      Mkt Value     Unreal P/L   Collateral
  ------------------------------------------------------------------------------------------------------------------------
  MU260227P00367500          -2 short  $   367.50 $       5.20 $         4.00 $      -800.00 $       240.00 $  73,500.00
  NEM260220P00110000         -5 short  $   110.00 $       0.61 $         0.15 $       -75.00 $       230.00 $  55,000.00
  SNDK260220P00525000        -1 short  $   525.00 $       2.67 $         0.15 $       -15.00 $       252.50 $  52,500.00
  WDC260227P00260000         -3 short  $   260.00 $       4.25 $         5.20 $    -1,560.00 $   

## Buy Test Shares

Randomly select tickers from the S&P 500 and buy 100 shares of each.

In [8]:
# Pick random tickers and buy shares
test_tickers = random.sample(config.ticker_universe, N_TEST_TICKERS)
print(f"Selected tickers: {test_tickers}")

buy_orders = []
for ticker in test_tickers:
    try:
        order_request = MarketOrderRequest(
            symbol=ticker,
            qty=SHARES_PER_TICKER,
            side=OrderSide.BUY,
            time_in_force=TimeInForce.DAY,
        )
        order = alpaca.trading_client.submit_order(order_request)
        print(f"  BUY {ticker}: {SHARES_PER_TICKER} shares — order {order.id} ({order.status.value})")
        buy_orders.append((ticker, str(order.id)))
    except Exception as e:
        print(f"  FAILED {ticker}: {e}")

print(f"\nSubmitted {len(buy_orders)} buy orders")

Selected tickers: ['VZ', 'EQR', 'EW']
  BUY VZ: 100 shares — order d927bb4a-4c65-437b-a717-34611fd56d80 (accepted)
  BUY EQR: 100 shares — order 3de3d5a2-068f-4f1c-84f6-e897e747d8d1 (accepted)
  BUY EW: 100 shares — order c0c9042f-baf6-4a4f-8ee6-d2c1702ff0ef (accepted)

Submitted 3 buy orders


In [9]:
# Wait for fills and verify
print("Waiting 5s for order fills...")
time.sleep(5)

positions = alpaca.trading_client.get_all_positions()
stock_positions = [
    p for p in positions
    if not (any(c.isdigit() for c in p.symbol) and len(p.symbol) > 10)
]

print(f"\nStock positions ({len(stock_positions)}):")
for p in stock_positions:
    side = p.side.value if hasattr(p.side, 'value') else str(p.side)
    print(f"  {p.symbol:<8} qty={float(p.qty):>6.0f}  side={side:<6} "
          f"avg_entry=${float(p.avg_entry_price):>8.2f}  current=${float(p.current_price):>8.2f}")

Waiting 5s for order fills...

Stock positions (0):


## CC Detection & Entry

Test the covered call manager's ability to:
1. Detect stock positions from Alpaca
2. Create wheel position entries
3. Select and sell covered calls

In [ ]:
# Step 1: Detect stock positions
new_count = await cc_manager._detect_stock_positions()
print(f"\nDetected {new_count} new stock position(s)")

In [ ]:
# Inspect wheel store
print("Wheel Positions:")
print("=" * 80)
for sym, pos in wheel_store.get_active().items():
    print(f"  {sym}:")
    print(f"    Shares:     {pos['shares']}")
    print(f"    Cost basis: ${pos['cost_basis']:.2f}")
    print(f"    Source:     {pos['source']}")
    print(f"    Status:     {pos['status']}")
    print(f"    CC rounds:  {pos['cc_rounds']}")
    print(f"    Current CC: {pos['current_cc']}")
    print()

In [ ]:
# Step 2: Run a full CC cycle (detection + entry)
current_vix = vix_fetcher.get_current_vix()
print(f"VIX: {current_vix:.2f}")
print()

cc_summary = await cc_manager.run_cycle(current_vix)
print(f"\nCC Cycle Summary: {cc_summary}")

In [ ]:
# Inspect results after CC entry
print("Wheel Positions (after CC entry):")
print("=" * 80)
for sym, pos in wheel_store.get_active().items():
    print(f"  {sym}:")
    print(f"    Status:      {pos['status']}")
    print(f"    Cost basis:  ${pos['cost_basis']:.2f}")
    print(f"    CC premiums: ${pos['total_cc_premiums']:.2f}")
    cc = pos.get('current_cc')
    if cc:
        print(f"    Active CC:   {cc['option_symbol']}")
        print(f"      Strike:    ${cc['strike']:.2f}")
        print(f"      Premium:   ${cc['entry_premium']:.2f}")
        print(f"      Qty:       {cc['quantity']}")
        print(f"      Expiry:    {cc['expiration']}")
    else:
        print(f"    Active CC:   None (no contracts available?)")
    print()

## Monitoring & Cleanup

In [ ]:
# Run another cycle to exercise monitoring (should show CCs as healthy)
current_vix = vix_fetcher.get_current_vix()
print(f"VIX: {current_vix:.2f}")
print()

cc_summary = await cc_manager.run_cycle(current_vix)
print(f"\nMonitoring Cycle Summary: {cc_summary}")

In [ ]:
# Cleanup: liquidate everything
print("Liquidating all holdings...")
liquidate_all_holdings(alpaca)

# Clear test wheel store
for sym in list(wheel_store.positions.keys()):
    wheel_store.terminate(sym, reason="test_cleanup")
print("\nWheel store cleaned up")
print("Done!")